# Google Cloud CMIP6 Public Data: Basic Python Example

https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/basic_search_and_load.html

This notebooks shows how to query the catalog and load the data using python

The columns of the dataframe correspond to the CMI6 controlled vocabulary. A beginners' guide to these terms is available in [this document](https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q). 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec
import geopandas as gpd
import os
import time

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
plt.rcParams['figure.figsize'] = 12, 6

/Users/maoyabassiouni/opt/anaconda3/envs/basePy3.9/lib/python3.11/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
data_dir = '/Users/maoyabassiouni/Documents/DATA/Projects/OLNI'

In [3]:
def get_df_CMIP(df_catalogue, query_string, print_q=None):
    df_q = df_catalogue.query(query_string)
    print(query_string, len(df_q.index))
    print('')
    if print_q:
        print(df_q[['source_id', 'experiment_id', 'table_id', 'variable_id', 'version', 'zstore']])
        print(list(set(df_q['source_id'].values)))
    
    # get the path to a specific zarr store (latest version if more than one)
    zstore = df_q.zstore.values[-1]
    print(zstore)
    print('')
    # create a mutable-mapping-style interface to the store
    mapper = fsspec.get_mapper(zstore)

    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    ds
    return ds

In [4]:
def make_loc_df(dsi, plot_locs, variable_name):
    [variable_id, experiment_id, source_id] = variable_name.split('_')
    data = np.array([])
    time_f = dsi.time.values#.astype("float64")
    lat_i, lon_i, sp_i = plot_locs[0]
    vals = dsi[variable_id].sel(lat=lat_i, lon =lon_i, method='nearest').values.astype("float64")
    ll = len(vals)
    print(lat_i, lon_i, sp_i, ll)
    data = np.vstack([np.ones(ll) * lat_i, 
               np.ones(ll) * lon_i, 
               np.ones(ll) * sp_i, 
               vals]).T 
    time_ff = time_f 
    for lat_i, lon_i, sp_i in plot_locs[1:]:
        vals = dsi[variable_id].sel(lat=lat_i, lon =lon_i, method='nearest').values.astype("float64")
        ll = len(vals)
        #print(lat_i, lon_i, sp_i, ll)
        data_i = np.vstack([np.ones(ll) * lat_i, 
                   np.ones(ll) * lon_i, 
                   np.ones(ll) * sp_i, 
                   vals]).T 
        data = np.concatenate([data, data_i])
        time_ff = np.concatenate([time_ff, time_f])
    lat, lon, sp, values = zip(*data)
    df_v = pd.DataFrame({'Lat': lat, 'Lon': lon, 'time': time_ff, 'sp': sp, variable_name: values})

    return df_v

In [5]:
df_catalogue = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_catalogue.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [6]:
scenarios = [['CMIP', 'historical'], ['ScenarioMIP', 'ssp245'],['ScenarioMIP', 'ssp585']]

variables_A = [['tas', 'Amon'], # Near-Surface Air Temperature [K]
             ['hurs', 'Amon'], #Near-Surface Relative Humidity [%]
             ['pr', 'Amon'], #Precipitation [kg m-2 s-1]
             ['rsds', 'Amon'], #Surface Downwelling Shortwave Radiation [W m-2]
             ]

variables_B = [['evspsblpot', 'Emon'], # Potential Evapotranspiration [kg m-2 s-1]
             ['co2mass','Amon'], #Total Atmospheric Mass of CO2 [kg]
             ['co2s', 'Emon'],  # Atmosphere CO2 [1e-06]
             ]

## finding models in catalogue with required variables

In [7]:
print('variables', variables_A)
print('')

models_set_all = []
for variable_id, table_id in variables_A:
    print(variable_id, table_id)
    models_v = []
    for activity_id, experiment_id in scenarios:
        query_string = "activity_id== '%s' & table_id == '%s' & variable_id == '%s' \
                        & experiment_id == '%s'"  \
                        % (activity_id, table_id, variable_id, experiment_id)

        df_q = df_catalogue.query(query_string)
        models = list(set(df_q['source_id'].values))
        models_v.append(models)
        print('\t', experiment_id, len(models))
        #print(models[:6])
    models_set = []
    for m in models_v[0]:
 
        mi = [1 for i in models_v[1:] if m in i]
        if len(mi)==(len(scenarios)-1):
            models_set.append([m, len(mi)+1])
    models_set_all.append(models_set)
    print('n set models', len(models_set))
    print('')
    
models_select = []
for m, nm in models_set_all[0]:
    mi = [1 for i in models_set_all[1:] if m in list(zip(*i))[0]]
    if len(mi)==(len(variables_A)-1):
        models_select.append([m, len(mi)+1])
print('Selected models', len(models_select))
print(list(zip(*models_select))[0])

variables [['tas', 'Amon'], ['hurs', 'Amon'], ['pr', 'Amon'], ['rsds', 'Amon']]

tas Amon
	 historical 64
	 ssp245 46
	 ssp585 47
n set models 46

hurs Amon
	 historical 54
	 ssp245 38
	 ssp585 40
n set models 37

pr Amon
	 historical 63
	 ssp245 46
	 ssp585 46
n set models 44

rsds Amon
	 historical 57
	 ssp245 39
	 ssp585 41
n set models 39

Selected models 32
('INM-CM4-8', 'MIROC-ES2L', 'IITM-ESM', 'NorESM2-MM', 'GFDL-ESM4', 'CESM2-WACCM', 'GFDL-CM4', 'FGOALS-g3', 'EC-Earth3-Veg', 'CNRM-ESM2-1', 'CanESM5-CanOE', 'IPSL-CM6A-LR', 'CMCC-CM2-SR5', 'CESM2', 'CNRM-CM6-1', 'KIOST-ESM', 'ACCESS-CM2', 'EC-Earth3-Veg-LR', 'CanESM5', 'HadGEM3-GC31-LL', 'MPI-ESM1-2-LR', 'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'FIO-ESM-2-0', 'CNRM-CM6-1-HR', 'EC-Earth3', 'FGOALS-f3-L', 'MIROC6', 'MRI-ESM2-0', 'GISS-E2-1-G', 'INM-CM5-0', 'KACE-1-0-G')


In [8]:

models_set_all = []
for variable_id, table_id in variables_B:
    print(variable_id, table_id)
    models_v = []
    for activity_id, experiment_id in scenarios:
        query_string = "activity_id== '%s' & table_id == '%s' & variable_id == '%s' \
                        & experiment_id == '%s'"  \
                        % (activity_id, table_id, variable_id, experiment_id)

        df_q = df_catalogue.query(query_string)
        models = list(set(df_q['source_id'].values))
        models_v.append(models)
        print('\t', experiment_id, len(models))
        #print(models[:6])
    models_set = []
    for m in models_v[0]:
 
        mi = [1 for i in models_v[1:] if m in i]
        if len(mi)==(len(scenarios)-1):
            models_set.append([m, len(mi)+1])
    models_set_all.append(models_set)
    print('n set models', len(models_set))
    print(models_set)
    

evspsblpot Emon
	 historical 6
	 ssp245 4
	 ssp585 5
n set models 3
[['CNRM-ESM2-1', 3], ['IPSL-CM6A-LR', 3], ['CNRM-CM6-1', 3]]
co2mass Amon
	 historical 17
	 ssp245 2
	 ssp585 2
n set models 2
[['GFDL-ESM4', 3], ['GFDL-CM4', 3]]
co2s Emon
	 historical 1
	 ssp245 1
	 ssp585 1
n set models 1
[['GFDL-ESM4', 3]]


## Extract loc data

In [9]:
df_ts_ = pd.read_csv(os.path.join(data_dir ,'model_inputs', 'df_model_all_annual.csv'))
df_ts_ = df_ts_[(df_ts_['yrange']>=10) & (df_ts_['count']>=6) 
                & (df_ts_['MODIS_Lai'].notna()) & (df_ts_['SATPSI'].notna())
                & (df_ts_['LNC'].notna())]
df_ts_ = df_ts_[(df_ts_['year']>=1995) & (df_ts_['year']<=2016)]

plot_locs = df_ts_.groupby(['Lat', 'Lon', 'sp']).mean().index
plot_locs

MultiIndex([(36.3706, -5.5783,  54.0),
            (37.0314,  -3.015,  46.0),
            (37.1553, -6.7325, 131.0),
            (37.9017, -2.8986, 129.0),
            (37.9089, 13.4042,  41.0),
            (38.4272, 16.1797,  20.0),
            (38.4736, -6.7578,  46.0),
            (38.4764, -2.4556, 130.0),
            (38.5197, -0.6314, 125.0),
            (38.8461,  21.305,  46.0),
            ...
            (65.8836,    13.8, 118.0),
            (65.9688, 23.6717, 118.0),
            (66.2986, 29.4625, 118.0),
            (66.3025, 29.5011, 118.0),
            (66.3325, 26.6456, 118.0),
            (66.3631, 26.7336, 134.0),
            (67.3356, 26.6489, 134.0),
            (67.9503, 24.0572, 134.0),
            (67.9994, 24.2361, 118.0),
            (69.7389, 26.9581, 134.0)],
           names=['Lat', 'Lon', 'sp'], length=409)

In [10]:
df_area = df_catalogue.query("variable_id == 'areacella' & source_id == 'GFDL-ESM4'")
ds_area = xr.open_zarr(fsspec.get_mapper(df_area.zstore.values[0]), consolidated=True)
plot_locs_cmip = []
for lat_i, lon_i, sp_i in plot_locs:
    ds_area_i = ds_area.sel(lat=lat_i, lon =lon_i, method='nearest')
    lon_c = ds_area_i['lon'].values.item()  
    lat_c = ds_area_i['lat'].values.item()  
    #lon_c = ds_area.lon.sel(lat=lat_i, lon =lon_i, method='nearest').values
    plot_locs_cmip.append('%5.4f_%5.4f_%d' %(lat_c, lon_c, sp_i))

print(len(plot_locs_cmip))
plot_locs_cmip_set = list(set(plot_locs_cmip))
plot_locs_cmip_set = [loc.split('_') for loc in plot_locs_cmip_set]
plot_locs_cmip_set = [[float(l) for l in loc] for loc in plot_locs_cmip_set]
print(len(plot_locs_cmip_set))

409
312


## Extract climate timeseries by model

In [11]:
models = [##'ACCESS-CM2',
          ##'CanESM5',
          ##'CESM2',
          #'CESM2-WACCM',
          #'CMCC-CM2-SR5',
          ##'CNRM-CM6-1',
          #'CNRM-CM6-1-HR',
          #'CNRM-ESM2-1',
          #'CanESM5-CanOE',
          #'EC-Earth3',
          #'EC-Earth3-Veg',
          ##'EC-Earth3-Veg-LR',
          #'FGOALS-f3-L',
          ##'FGOALS-g3',
          #'FIO-ESM-2-0',
          ##'GFDL-CM4',
          #'GFDL-ESM4',
          ##'GISS-E2-1-G',
          #'HadGEM3-GC31-LL',
          ##'IITM-ESM',
          #'INM-CM4-8',
          ##'INM-CM5-0',
          ##'IPSL-CM6A-LR',
          #'KACE-1-0-G',
          ##'KIOST-ESM',
          #'MIROC-ES2L',
          ##'MIROC6',
          #'MPI-ESM1-2-HR',
          'MPI-ESM1-2-LR',
          'MRI-ESM2-0',
          'NorESM2-MM',
          'UKESM1-0-LL']

In [12]:
variables_A = variables_A[:2]
variables_A

[['tas', 'Amon'], ['hurs', 'Amon']]

In [13]:
#source_id = 'CESM2'
locations = plot_locs
#locations = plot_locs_cmip_set
activity_id = 'ScenarioMIP'
experiment_id = 'ssp245'
#for activity_id, experiment_id in scenarios:
for source_id in models:
    for variable_id, table_id in variables_A:
        variable_name = '%s_%s_%s' % (variable_id, experiment_id, source_id)
        print(variable_name, '.....................................................')
        print('.............................................................................................')
        query_string = "activity_id== '%s' & table_id == '%s' & variable_id == '%s' \
                        & experiment_id == '%s' & source_id == '%s'" \
                        % (activity_id, table_id, variable_id, experiment_id, source_id)

        dsi = get_df_CMIP(df_catalogue, query_string)
        df_v = make_loc_df(dsi, locations, variable_name)
        df_v.to_csv(os.path.join(data_dir, 'grid_data_extract/CMIP', '%s.csv' % (variable_name))) 


tas_ssp245_MPI-ESM1-2-LR .....................................................
.............................................................................................
activity_id== 'ScenarioMIP' & table_id == 'Amon' & variable_id == 'tas'                         & experiment_id == 'ssp245' & source_id == 'MPI-ESM1-2-LR' 10

gs://cmip6/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp245/r9i1p1f1/Amon/tas/gn/v20190710/

36.3706 -5.5783 54.0 1032
hurs_ssp245_MPI-ESM1-2-LR .....................................................
.............................................................................................
activity_id== 'ScenarioMIP' & table_id == 'Amon' & variable_id == 'hurs'                         & experiment_id == 'ssp245' & source_id == 'MPI-ESM1-2-LR' 10

gs://cmip6/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp245/r4i1p1f1/Amon/hurs/gn/v20190815/

36.3706 -5.5783 54.0 1032
tas_ssp245_MRI-ESM2-0 .....................................................
...........................

## Extract CO2 timeseries

In [52]:
min_lat = np.min(list(zip(*plot_locs_cmip_set))[0])
max_lat = np.max(list(zip(*plot_locs_cmip_set))[0])
min_lon = np.min(list(zip(*plot_locs_cmip_set))[1])
max_lon = np.max(list(zip(*plot_locs_cmip_set))[1])

df_area = df_catalogue.query("variable_id == 'areacella' & source_id == 'GFDL-ESM4'")
ds_area = xr.open_zarr(fsspec.get_mapper(df_area.zstore.values[0]), consolidated=True)
ds_area = ds_area.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon))
total_area = ds_area.areacella.sum(dim=['lon', 'lat'])

source_id = 'GFDL-ESM4'
table_id = 'Emon'
variable_id = 'co2s'
for activity_id, experiment_id in scenarios: 
    variable_name = '%s_%s_%s' % (variable_id, experiment_id, source_id)
    print(variable_name)
    query_string = "activity_id== '%s' & table_id == '%s' & variable_id == '%s' \
                            & experiment_id == '%s' & source_id == '%s'" \
                            % (activity_id, table_id, variable_id, experiment_id, source_id)

    dsi = get_df_CMIP(df_catalogue, query_string)
    dsi = dsi.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon))
    co2_timeseries = (dsi[variable_id] * ds_area.areacella).sum(dim=['lon', 'lat']) / total_area
    
    co2_y_timeseries = co2_timeseries.resample(time='AS').mean('time')
    co2_df = pd.DataFrame({'datetime': co2_y_timeseries.time, 'CO2_%s' % experiment_id: co2_y_timeseries.values})
    co2_df.to_csv(os.path.join(data_dir, 'grid_data_extract/CMIP', '%s.csv' % (variable_name))) 

co2s_historical_GFDL-ESM4
activity_id== 'CMIP' & table_id == 'Emon' & variable_id == 'co2s'                             & experiment_id == 'historical' & source_id == 'GFDL-ESM4' 1

gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Emon/co2s/gr1/v20190726/
co2s_ssp245_GFDL-ESM4
activity_id== 'ScenarioMIP' & table_id == 'Emon' & variable_id == 'co2s'                             & experiment_id == 'ssp245' & source_id == 'GFDL-ESM4' 1

gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp245/r1i1p1f1/Emon/co2s/gr1/v20180701/
co2s_ssp370_GFDL-ESM4
activity_id== 'ScenarioMIP' & table_id == 'Emon' & variable_id == 'co2s'                             & experiment_id == 'ssp370' & source_id == 'GFDL-ESM4' 1

gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp370/r1i1p1f1/Emon/co2s/gr1/v20180701/
co2s_ssp585_GFDL-ESM4
activity_id== 'ScenarioMIP' & table_id == 'Emon' & variable_id == 'co2s'                             & experiment_id == 'ssp585' & source_id == 'GFDL-ESM4' 1

gs://cmi

In [13]:
df_area = df_catalogue.query("variable_id == 'areacella' & source_id == 'CESM2'")
ds_area = xr.open_zarr(fsspec.get_mapper(df_area.zstore.values[0]), consolidated=True)

In [ ]:
query_string = "activity_id=='ScenarioMIP' \
                & table_id == 'Amon' \
                & variable_id == 'tas' \
                & experiment_id == 'ssp245'\
                & institution_id == 'NCAR'"
variable_id = 'tas'
dsi = get_df_CMIP(df_catalogue, query_string, print_q=True)

In [ ]:
dsi[variable_id].sel(time='2050-01').squeeze().plot()

In [ ]:
total_area = ds_area.areacella.sum(dim=['lon', 'lat'])
total_area

In [ ]:
ta_timeseries = (dsi[variable_id] * ds_area.areacella).sum(dim=['lon', 'lat']) / total_area
ta_timeseries

In [ ]:
%time ta_timeseries.load()
ta_timeseries.plot(label='monthly')
ta_timeseries.rolling(time=12).mean().plot(label='12 month rolling mean')
plt.legend()


In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world[world['continent']=='Europe']
print(europe['name'].values)
x_names = ['Russia','Iceland']
europe_sel = europe
for n in x_names:
    europe_sel = europe_sel[europe_sel['name']!=n]
europe_sel.plot()